# Class 29 - Flux Balance Analysis

In this exercise we'll load a simplified metabolic network for the bacteria *Escherichia coli* and find the steady-state metabolic flux through this network that maximizes the rate of cellular growth, subject to thermodynamic constraints.

Read the stoichiometric matrix from a tab-delimited data file with M rows and N columns

In [11]:
S <- as.matrix(read.table("shared/ucsd_ecoli_stoich_matrix.txt",
                          header=TRUE,
                          row.names=1,
                          sep="\t",
                          comment.char="",
                          quote="",
                          stringsAsFactors=FALSE))
M <- nrow(S)
N <- ncol(S)
cat(sprintf("The stoichiometrix matrix has %d rows (metabolites) and %d columns (reactions)\n",
             M, N))

The stoichiometrix matrix has 72 rows (metabolites) and 95 columns (reactions)


Read the min and max flux values, and the objective function coefficients, from a tab-delimited data file with N rows and 3 columns

In [9]:
v_minmax <- as.matrix(read.table("shared/ucsd_ecoli_reaction_maxmin.txt",
                                 sep="\t",
                                 header=TRUE,
                                 comment.char="",
                                 row.names=1,
                                 quote="",
                                 stringsAsFactors=FALSE))
                      
stopifnot(all(make.names(rownames(v_minmax)) == colnames(S)))

head(v_minmax)

,min,max,objective
ACALD,-1000,1000,0
ACALDt,-1000,1000,0
ACKr,-1000,1000,0
ACONTa,-1000,1000,0
ACONTb,-1000,1000,0
ACt2r,-1000,1000,0


Create a constraint matrix (with C rows and N columns) and a corresponding constant vector of length C, giving the right-hand-side of the inequality

In [15]:
Svmin <- S %*% matrix(v_minmax[,"min"], ncol=1)

constraint_matrix <- rbind(S,
                           S,
                           diag(rep(1,N)),
                           diag(rep(1,N)))

constraint_rhs <- c(-Svmin,
                    -Svmin,
                    rep(0, N),
                    v_minmax[,"max"] - v_minmax[,"min"])

constraint_dir <- c(rep(">=", M),
                    rep("<=", M),
                    rep(">=", N),
                    rep("<=", N))

stopifnot(length(constraint_rhs) == nrow(constraint_matrix))
stopifnot(ncol(constraint_matrix) == N)
stopifnot(nrow(v_minmax) == N)

dim(constraint_matrix)

[1] 334  95

Find the flux value that maximizes the "biomass" reaction flux, consistent with the steady-state and thermodynamic constraints

In [16]:
library(lpSolve)
lpres <- lp("max",
   objective.in=v_minmax[,"objective"],
   const.mat=constraint_matrix,
   const.dir=constraint_dir,
   const.rhs=constraint_rhs)

print(lpres$solution + v_minmax[,"min"])

             ACALD             ACALDt               ACKr             ACONTa 
      0.000000e+00       1.136868e-13      -2.273737e-13       6.007250e+00 
            ACONTb              ACt2r               ADK1              AKGDH 
      6.007250e+00      -1.136868e-13       0.000000e+00       5.064376e+00 
            AKGt2r             ALCD2x               ATPM             ATPS4r 
     -1.136868e-13      -1.136868e-13       8.390000e+00       4.551401e+01 
Biomass_Ecoli_core               CO2t                 CS              CYTBD 
      8.739215e-01      -2.280983e+01       6.007250e+00       4.359899e+01 
           D-LACt2                ENO            ETOHt2r           EX_ac(e) 
     -1.136868e-13       1.471614e+01      -1.136868e-13       0.000000e+00 
       EX_acald(e)          EX_akg(e)          EX_co2(e)         EX_etoh(e) 
      0.000000e+00       0.000000e+00       2.280983e+01       0.000000e+00 
         EX_for(e)          EX_fru(e)          EX_fum(e)          EX_glc(e) 